# Marine Carbonate System Visualizer - TA & pH

## Interactive Bjerrum Plot for Marine Chemistry

<div style="font-size: 22px; line-height: 1.6;">

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

<div style="text-align: center; font-size: 18px; color: #00204C; margin: 20px 0;">
    <strong>Comprehensive visualization using Total Alkalinity and pH variations</strong>
</div>

</div>

This tool visualizes the marine carbonate system using **Bjerrum plots** with **Total Alkalinity (TA)** and **pH** as input parameters.

**Features:**
- Interactive sliders for Total Alkalinity and pH
- 4-panel visualization showing carbonate speciation
- Real-time calculations using PyCO2SYS
- Educational tool for marine chemistry courses

**Instructions:**
1. Adjust the sliders below to change Total Alkalinity and pH values
2. Observe how the carbonate system responds in real-time
3. Use the Reset button to return to default values

## What does this tool show?

**Input parameters:**
- **Total alkalinity** (TA) in μmol/kg
- **pH** (total scale)

**Fixed conditions:**
- **Salinity** = 35 PSU
- **Temperature** = 25°C

**Visualizations:**
1. **Bjerrum Plot** - Species fractions vs pH
2. **Current Composition** - Pie chart of species
3. **Saturation State** - Aragonite saturation
4. **System Analysis** - Detailed results

**Educational Value:**
- Understand how pH directly affects carbonate speciation
- Explore the relationship between alkalinity and pH
- Analyze the impact of pH changes on calcification
- Study buffering effects across different pH ranges

In [ ]:
# Load necessary libraries
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, Layout, HTML
from IPython.display import display

In [ ]:
# 🌊 Marine Carbonate System Configuration

# Standard seawater conditions for educational calculations
CONFIG = {
    'salinity': 35,              # Practical Salinity Units (typical ocean value)
    'temperature': 25,           # Temperature in °C (warm tropical water)
    'pressure': 0,               # Pressure in dbar (surface water)
    'opt_pH_scale': 1,          # pH scale: 1=Total, 2=SWS, 3=Free, 4=NBS
    'opt_k_carbonic': 10        # K1,K2 constants: 10=Waters2014 (recommended)
}

# Educational examples for students:
# set_config(temperature=15)    # Cooler water (affects equilibria)
# set_config(salinity=30)       # Lower salinity (coastal/estuarine)
# set_config(pressure=100)      # Deeper water (100m depth approx)

def set_config(**kwargs):
    """Update seawater conditions and recalculate carbonate system
    
    Students can use this function to explore how different conditions
    affect the carbonate system equilibria.
    
    Examples:
        set_config(temperature=15)  # Simulate polar waters
        set_config(salinity=30)     # Simulate coastal waters
    """
    CONFIG.update(kwargs)
    print(f"🔧 Updated configuration: {CONFIG}")
    print("🔄 Interactive widgets will use new conditions automatically")

print("🌊 Marine Carbonate System configured!")
print("📊 Standard conditions: 35 PSU, 25°C, surface water")
print("⚙️ Use set_config() to modify conditions for different scenarios")

# Common constants options for reference:
print("🔬 PyCO2SYS Configuration Ready!")
print(f"📋 Current settings: {CONFIG}")
print("\n📚 Common opt_k_carbonic options:")
print("   8  = Lueker et al. (2000) - Most common")
print("   10 = Waters et al. (2014) - Real seawater (default)")
print("   14 = Schockman & Byrne (2021) - Latest")
print("\n💡 Use: set_config(opt_k_carbonic=8) to change")

In [ ]:
# ⚗️ Carbonate System Calculation Functions

def get_pyco2_params(alkalinity, pH):
    """Get standardized PyCO2SYS parameters for TA-pH calculations"""
    return {
        'par1': alkalinity,         # Total Alkalinity (μmol/kg)
        'par2': pH,                # pH (total scale)
        'par1_type': 1,            # 1 = Total Alkalinity
        'par2_type': 3,            # 3 = pH
        'salinity': CONFIG['salinity'],
        'temperature': CONFIG['temperature'], 
        'pressure': CONFIG['pressure'],
        'opt_pH_scale': CONFIG['opt_pH_scale'],
        'opt_k_carbonic': CONFIG['opt_k_carbonic']
    }

def compute_carbonate_system(alkalinity, pH):
    """Compute carbonate system using PyCO2SYS with TA and pH as inputs"""
    try:
        # Run PyCO2SYS calculation
        results = pyco2.sys(**get_pyco2_params(alkalinity, pH))
        
        # Extract results using robust key access
        return {
            "pH_total": float(results["pH_total"]),
            "pCO2": float(results["pCO2"]), 
            "bicarbonate": float(results["bicarbonate"]),
            "carbonate": float(results["carbonate"]),
            "DIC": float(results["dic"]),  # PyCO2SYS uses lowercase 'dic'
            "omega_aragonite": float(results["saturation_aragonite"]),
            "alkalinity": float(results["alkalinity"]),
            **CONFIG
        }
        
    except KeyError as e:
        print(f"❌ KeyError: {e}")
        print("Available keys in PyCO2SYS results:")
        if 'results' in locals():
            available_keys = list(results.keys())
            print(f"   {available_keys[:10]}...")  # Show first 10 keys
        raise
        
    except Exception as e:
        print(f"❌ Error in carbonate system calculation: {e}")
        print(f"   Input: TA={alkalinity}, pH={pH}")
        print(f"   Config: {CONFIG}")
        raise

def get_constants_from_current_system(data):
    """Extract K1, K2 constants from current equilibrium state"""
    try:
        pH = data['pH_total']
        pCO2 = data['pCO2'] 
        HCO3 = data['bicarbonate']
        CO3 = data['carbonate']
        
        # Calculate [H+] and [CO2*]
        H = 10**(-pH)
        CO2_star = pCO2 * 0.034  # Henry's law constant for CO2 in seawater
        
        # Calculate equilibrium constants from concentrations
        # K1 = [H+][HCO3-] / [CO2*]
        # K2 = [H+][CO3-2] / [HCO3-]
        K1 = (H * HCO3 / CO2_star) if CO2_star > 0 else 1e-6
        K2 = (H * CO3 / HCO3) if HCO3 > 0 else 1e-9
        
        return K1, K2
        
    except Exception as e:
        print(f"❌ Error calculating constants: {e}")
        # Return default values if calculation fails
        return 1e-6, 1e-9

print("⚗️ TA-pH calculation functions ready!")
print("🎯 This tool shows direct pH effects on carbonate speciation")
print("🐚 Perfect for studying calcification and buffering capacity")

---

## 🚀 How to Use This Tool

### Step 1: Configure PyCO2SYS (Above)
- **Default settings** work for typical seawater conditions
- **To change**: Use `set_config(parameter=value)` in the configuration cell
- **Common changes**: 
  - `set_config(opt_k_carbonic=8)` - Use Lueker et al. (2000) constants
  - `set_config(temperature=15, salinity=30)` - Different conditions

### Step 2: Run the Interactive Interface (Below)
- Move the **sliders** to explore different TA and pH values
- **Plots update automatically** showing:
  1. **Bjerrum Plot** - Species fractions vs pH
  2. **Composition** - Current system breakdown  
  3. **Saturation** - Aragonite saturation state
  4. **Analysis** - Detailed results with your chosen constants

### Step 3: Analyze Results
- **Red line** in Bjerrum plot shows current pH
- **pH variations** directly show speciation changes
- **All calculations** use the same consistent parameters

### Educational Applications:
- **Direct pH Control**: See immediate effects of pH changes
- **Buffering Analysis**: How alkalinity affects pH sensitivity
- **Ocean Acidification**: Explore lower pH scenarios
- **Calcification Studies**: pH effects on saturation states

---

In [ ]:
# 📊 Plotting Functions - Bjerrum Diagram and Analysis

def create_bjerrum_plot(data):
    """Create 4-panel Bjerrum plot with system analysis for TA-pH"""
    try:
        plt.close('all')
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Marine Carbonate System Analysis (TA-pH)', fontsize=16, fontweight='bold')
        
        colors = ['#443983', '#31688e', '#35b779', '#fde725']
        pH_range = np.linspace(4, 11, 40)
        
        # Get constants from current system state
        K1, K2 = get_constants_from_current_system(data)
        
        # Calculate speciation using analytical expressions
        H_values = 10**(-pH_range)
        denom = H_values**2 + K1*H_values + K1*K2
        
        # Ensure denominator is never zero
        denom = np.where(denom == 0, 1e-15, denom)
        
        alpha0 = H_values**2 / denom      # CO2
        alpha1 = K1*H_values / denom      # HCO3-
        alpha2 = K1*K2 / denom            # CO3-2
        
        # Plot 1: Bjerrum diagram
        ax1.plot(pH_range, alpha0, color=colors[0], linewidth=3, label='CO2')
        ax1.plot(pH_range, alpha1, color=colors[1], linewidth=3, label='HCO3-')
        ax1.plot(pH_range, alpha2, color=colors[2], linewidth=3, label='CO3-2')
        
        current_pH = data['pH_total']
        ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=2)
        ax1.text(current_pH + 0.1, 0.8, f'pH: {current_pH:.2f}', fontsize=10, color='red')
        ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction of DIC')
        ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
        ax1.set_xlim(4, 11), ax1.set_ylim(0, 1)
        
        # Plot 2: Composition pie chart
        CO2_conc = data["pCO2"] * 0.034  # Henry's law
        species = [CO2_conc, data["bicarbonate"], data["carbonate"]]
        species_labels = ['CO2*', 'HCO3-', 'CO3-2']
        
        ax2.pie(species, labels=species_labels, autopct='%1.1f%%', colors=colors[:3])
        ax2.set_title(f'Composition at pH {current_pH:.2f}')
        
        # Plot 3: Saturation state
        omega = data["omega_aragonite"]
        color_omega = colors[3] if omega >= 1 else colors[1]
        ax3.bar(['Aragonite'], [omega], color=color_omega)
        ax3.axhline(y=1, color='black', linestyle='--', alpha=0.7)
        ax3.set_ylabel('Omega (Ω)'), ax3.set_title('Saturation State')
        
        # Add saturation status
        status = "Supersaturated" if omega >= 1 else "Undersaturated"
        ax3.text(0, omega*0.7, f'Ω = {omega:.2f}\n{status}', ha='center', fontsize=11)
        
        # Plot 4: System information
        ax4.axis('off')
        
        # Calculate display values
        TA_calc = data.get('alkalinity', data['bicarbonate'] + 2*data['carbonate'])
        DIC_calc = data['DIC']
        
        # Use text without problematic emojis for matplotlib compatibility
        info_text = f"""INPUTS:
TA = {TA_calc:.0f} umol/kg
pH = {current_pH:.2f}

RESULTS:
DIC = {DIC_calc:.0f} umol/kg
pCO2 = {data['pCO2']:.0f} uatm
[HCO3-] = {data['bicarbonate']:.0f} umol/kg
[CO3-2] = {data['carbonate']:.0f} umol/kg
Omega_arag = {omega:.2f}

CONSTANTS:
K1 = {K1:.2e}
K2 = {K2:.2e}

CONDITIONS:
S = {data['salinity']} PSU
T = {data['temperature']}°C
pH scale = {data['opt_pH_scale']}
K constants = {data['opt_k_carbonic']}

INTERPRETATION:
pH controls speciation directly
TA controls buffering capacity
Current composition shown above"""
        
        ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes, fontsize=9,
                 verticalalignment='top', fontfamily='monospace')
        
        plt.tight_layout()
        plt.show()
        return fig
        
    except Exception as e:
        print(f"Error creating plot: {e}")
        print(f"   Data keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
        raise

print("📊 Robust plotting functions ready for TA-pH analysis!")
print("🛡️ Enhanced error handling and data validation")

In [ ]:
# 🎛️ Interactive Interface - TA and pH Sliders

# Create sliders
sliders = {
    'alk': FloatSlider(value=2300, min=1800, max=3500, step=50,
                      description="Alkalinity (μmol/kg):", 
                      style={'description_width': 'initial'}, layout=Layout(width='600px')),
    'ph': FloatSlider(value=8.1, min=6.5, max=8.8, step=0.05,
                     description="pH (total scale):",
                     style={'description_width': 'initial'}, layout=Layout(width='600px'))
}

# Update function (no duplicates)
def update_plots(alkalinity, pH):
    """Update plot when sliders change"""
    data = compute_carbonate_system(alkalinity, pH)
    create_bjerrum_plot(data)

# Reset function
def reset_values(b):
    """Reset sliders to default values"""
    sliders['alk'].value, sliders['ph'].value = 2300, 8.1

# Preset buttons for common scenarios
def set_high_pH(b):
    """Set to high pH conditions (low CO2)"""
    sliders['ph'].value = 8.5
    
def set_normal_pH(b):
    """Set to normal ocean pH"""
    sliders['ph'].value = 8.1
    
def set_low_pH(b):
    """Set to acidified conditions"""
    sliders['ph'].value = 7.7

# Create buttons
reset_button = Button(description="Reset Values", button_style='info')
high_pH_button = Button(description="High pH (8.5)", button_style='success')
normal_pH_button = Button(description="Normal Ocean pH (8.1)", button_style='warning')
low_pH_button = Button(description="Acidified Ocean (7.7)", button_style='danger')

# Connect buttons to functions
reset_button.on_click(reset_values)
high_pH_button.on_click(set_high_pH)
normal_pH_button.on_click(set_normal_pH)
low_pH_button.on_click(set_low_pH)

# Create button layout
button_box = VBox([
    HTML("<h4>Quick pH Scenarios:</h4>"),
    VBox([normal_pH_button, high_pH_button, low_pH_button, reset_button], 
         layout=Layout(width='100%'))
])

# Display complete interface
display(VBox([
    HTML("<h3>Marine Carbonate System Explorer - TA & pH</h3>"),
    HTML("<p><em>Direct pH control to explore carbonate chemistry changes</em></p>"),

    sliders['alk'], 
    sliders['ph'],
    button_box,
    interactive_output(update_plots, {'alkalinity': sliders['alk'], 'pH': sliders['ph']})
]))

print("Interactive TA-pH carbonate system interface ready!")
print("All calculations use PyCO2SYS with your chosen constants")
print("Use the scenario buttons to explore different pH conditions")
print("Normal ocean pH ≈ 8.1")
print("Ocean acidification: pH decreasing to 7.7-7.8")
print("High pH scenarios: 8.3-8.5 (low CO2 conditions)")

**Author:** Cardoso-Mohedano JG  
**Institution:** Instituto de Ciencias del Mar y Limnologia, UNAM, Estacion El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.

### Marine pH & Alkalinity Educational Notes:

**pH Ranges in Marine Environments:**
- **High pH (8.3-8.5)**: Low CO2, high carbonate ion concentration
- **Normal Ocean (8.0-8.2)**: Current typical ocean surface pH
- **Acidified Ocean (7.7-7.9)**: Ocean acidification scenarios

**Key Learning Objectives:**
1. **Direct pH Effects**: Immediate impact on carbonate speciation
2. **Alkalinity Buffering**: How TA provides resistance to pH change
3. **Calcification Impact**: pH effects on shell/coral formation
4. **System Relationships**: Connection between pH, DIC, and pCO2